In [1]:
import math
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import collections as col
import re
import random
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
#from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
#from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import auc,roc_curve

In [2]:
def load_data(train='Yes', test='No', validation='No'):
	"""Loads and returns datasets as required
	   Return empty lst for if 'No'
	"""
	if train=='Yes':
		df_train = pd.read_csv('dataset/train.csv', sep=',')
	else:
		df_train = []

	if test=='Yes':
		df_test = pd.read_csv('dataset/test.csv', sep=',')
	else:
		df_test = []

	if validation=='Yes':
		df_validation = pd.read_csv('dataset/validation.csv', sep=',')
	else:
		df_validation = []
	
	print('Data loaded', len(df_train), len(df_test), len(df_validation))
	return df_train, df_test, df_validation

In [107]:
df_train, df_test, df_validation= load_data('No', 'Yes', 'No')

Data loaded 0 299749 0


In [95]:
average = 0.0007539649884458758

In [104]:
df_temp= pd.DataFrame()
df_temp= pd.read_csv('pCTR/pCTR_test_82.csv')
df_temp.columns = ['index', 'pCTR']

df_model= pd.DataFrame()
df_model = df_temp
#df_model['pCTR'] = df_temp['pCTR_82']
df_model['pCTR_norm'] = df_model['pCTR'].values/ average

#df_model= df_model.drop(['weekday', 'hour', 'logtype', 'userid', 'useragent',
#       'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid',
#       'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
#       'slotprice', 'creative', 'keypage',
#       'advertiser', 'usertag'], axis=1)



In [108]:
df_model['bidid']= df_test['bidid']

In [80]:
def constant_bid(baseline_bid, pCTR_norm= None, c= None, lambd= None):
    con_bid= baseline_bid
    return con_bid

def random_bid(baseline_bid, pCTR_norm= None, c= None, lambd= None):
    ran_bid= np.random.randint(-1, baseline_bid)
    return ran_bid

def linear_bid(baseline_bid, pCTR_norm, c= None, lambd= None):
    lin_bid= baseline_bid* (pCTR_norm)
    return lin_bid

def linear_bid_2(baseline_bid, pCTR_norm, c= None, lambd= None):
    lin_bid= baseline_bid* (pCTR_norm** 2)
    return lin_bid

def linear_bid_3(baseline_bid, pCTR_norm, c= None, lambd= None):
    lin_bid= baseline_bid* (pCTR_norm** 3)
    return lin_bid

def linear_alt_2(baseline_bid, pCTR_norm, c= None, lambd= None):
    if pCTR_norm> 30:
        lin_bid= 300
    else:
        lin_bid= baseline_bid* (pCTR_norm** 2)
    return lin_bid

def linear_alt_3(baseline_bid, pCTR_norm, c= None, lambd= None):
    if pCTR_norm> 30:
        lin_bid= 300
    else:
        lin_bid= baseline_bid* (pCTR_norm** 3)
    return lin_bid

def linear_alt_5(baseline_bid, pCTR_norm, c= None, lambd= None):
    if pCTR_norm> 30:
        lin_bid= 300
    else:
        lin_bid= baseline_bid* (pCTR_norm** 5)
    return lin_bid

def linear_alt_7(baseline_bid, pCTR_norm, c= None, lambd= None):
    if pCTR_norm> 20:
        lin_bid= 300
    else:
        lin_bid= baseline_bid* (pCTR_norm)
    return lin_bid

def w_bid(baseline_bid, pCTR_norm, c, lambd):
    w_b= b_bid(baseline_bid, pCTR_norm, c, lambd)/ (b_bid(baseline_bid, pCTR_norm, c, lambd)+ c)
    return w_b

def b_bid(baseline_bid, pCTR_norm, c, lambd):
    nlin_bid= (np.sqrt(((c/ lambd)* pCTR_norm)+ (c**2))- c)/ (\
              (np.sqrt(((c/ lambd)* pCTR_norm)+ (c**2))- c)+ c)* baseline_bid
    return nlin_bid

In [184]:
def run_model(df_model, baseline_bid, bid_strat, c= 20, lambd= 0.0001, test_run= False, save_run= True):
    
    if test_run== False:
        win_count= 0
        win_click_count= 0
        loss_count= 0
        loss_click_count= 0
        total_cost= 0
        budget= 250000
        loss_imp= {}
        win_imp= {}

        for index, row in df_model.iterrows():
            if row['payprice']<= bid_strat(baseline_bid, row['pCTR_norm'], c, lambd):
                win_count+= 1
                if row['click']== 1:
                    win_click_count+= 1
                    win_imp[index]= (row['pCTR_norm'], row['payprice'])

                total_cost+= row['payprice']
                if total_cost> budget:
                    break

            else:
                loss_count+= 1
                if row['click']== 1:
                    loss_click_count+= 1

                    loss_imp[index]= (row['pCTR_norm'], row['payprice'])

        try:
            CTR= (win_click_count/ win_count)
        except:
            CTR= 0
            
        return [bid_strat.__name__, baseline_bid, win_count, win_click_count, loss_count, loss_click_count, CTR, total_cost], \
loss_imp, win_imp

    if test_run== True:
        
        lst_bid= []

        for index, row in df_model.iterrows():
            lst_bid.append(linear_alt_3(baseline_bid, row['pCTR_norm'], 0, 0))

        df_model['bid']= lst_bid
        
        df_output = pd.DataFrame()
        df_output[['bidid', 'bidprice']]= df_model[['bidid', 'bid']]

        if save_run== True:
            output_directory= 'Submission/Val/'
            output_filename= 'Group_22.csv'
            df_output.to_csv((output_directory + output_filename), index= False)
            print('Submission file saved: ', os.getcwd(), output_directory, output_filename)
        if save_run== False:
            print('Submission file not saved')
        
        return df_output

In [183]:
df_output = run_model(df_model, 0, linear_alt_3, c= 20, lambd= 0.0001, test_run=True, save_run=True)

Submission file saved:  /Users/jamesshields/MSc-Data-Science/WE Submission/Val/ Group_XX.csv


In [92]:
try:
    df_results
except:
    df_results= pd.DataFrame()

for i in range(1):
    results, loss_imp, win_imp= run_model(df_validation, i, linear_alt_3)
    if len(df_results)== 0:
        df_results = pd.DataFrame([results], columns=('bid_strat', 'baseline_bid', 'win_count', 'win_click_count', 'loss_count', 'loss_click_count', 'CTR', 'total_cost'))
    else:
        df_temp = pd.DataFrame()
        df_temp = pd.DataFrame([results], columns=('bid_strat', 'baseline_bid', 'win_count', 'win_click_count', 'loss_count', 'loss_click_count', 'CTR', 'total_cost'))
        df_results= df_results.append(df_temp)

In [94]:
df_results[df_results['bid_strat']== 'linear_alt_3']

,bid_strat,baseline_bid,win_count,win_click_count,loss_count,loss_click_count,CTR,total_cost
0,linear_alt_3,0,1179,26,298570,200,0.022053,115470
0,linear_alt_3,1,3591,33,75528,37,0.009190,250014
0,linear_alt_3,2,3529,28,59334,30,0.007934,250081
0,linear_alt_3,0,1179,26,298570,200,0.022053,115470
0,linear_alt_3,0,1249,26,298500,200,0.020817,115470


In [130]:
linear_alt_3(0, df_model['pCTR_norm'].values, 0, 0)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [3]:
import pandas as pd

def load_data():
	print('Importing data')
	df_x = pd.DataFrame()
	df_x = df_x.from_csv("X_train_features2.csv")
	df_x.drop('usertag', axis=1, inplace=True)
	x_array = df_x.as_matrix()

	df_y = pd.DataFrame()
	df_y= df_y.from_csv("Y_train_labels.csv")
	y_array = df_y.as_matrix()

	print('Datsa imported', x_array.shape, y_array.shape)
	return x_array, y_array

In [4]:
x_array, y_array= load_data()

Importing data
Datsa imported (2697738, 151) (2697738, 1)
